In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from pathlib import Path

from datetime import datetime

In [43]:
ac_data = "C:\\Users\\Laevskiy\\Desktop\\05-22\\merge102sortedbydate.xlsx"
chem_in = "C:\\Users\\Laevskiy\\Desktop\\05-22\\!chem баки питания.xlsx"
chem_out = "C:\\Users\\Laevskiy\\Desktop\\05-22\\!chem окисленная пульпа.xlsx"
chem_fe2 = "C:\\Users\\Laevskiy\\Desktop\\05-22\\!chem Fe2+.xlsx"

In [83]:
WORK_DIR = "C:\\Users\\Laevskiy\\Desktop\\05-22"

In [84]:
def file_save(df: pd.DataFrame, folder: str, name: str) -> None:
    comp = {
        'method': 'zip',
        'archive_name': 'out.csv'
    }

    filepath = Path(f'./{folder}/{name}.zip')

    df.to_csv(filepath, compression=comp)

## Подготовка данных

### Данные АСУТП

In [46]:
df_ac = pd.read_excel(ac_data, index_col=1, parse_dates=True)
df_ac.drop("Unnamed: 0", axis=1, inplace=True)

In [47]:
df_ac.isna().sum()

320PU102              17
320PU221              17
320RC102DIC02         17
330AC102FI06          17
330AC102FI07          17
330AC102FI08          17
330AC102FI09          17
330AC102FI10          17
330AC102FI51          17
330AC102FI52          17
330AC102FIC40         17
330AC102FICZ01        17
330AC102FICZ02        17
330AC102FICZ03        17
330AC102FICZ04        17
330AC102FICZ05        17
330AC102LICZ01        17
330AC102LICZ01.1      17
330AC102LICZ01.2      17
330AC102PI64          17
330AC102PI65         933
330AC102PIC63A        17
330AC102PIC63A.1      17
330AC102PIZ61         17
330AC102TIC01         17
330AC102TIC02         17
330AC102TIC03         17
330AC102TIC04         17
330AC102TIC05         17
330AC102TIZ61         17
330PT103PIZ01         17
330TK103LIC01         17
330TK103PI11          17
330TK103TI03          17
330TK103TIC02         17
330TK104LIC01         17
330TK104PI02          17
330TK104TIC02         17
380HX103FIC01         17
380HX103LIC01         17


Уберем пропуски

In [48]:
df_ac.dropna(inplace=True)

Усредним все данные с 10 минутных до часовых. Для дальнейшей стыковки с данными ХА.

In [49]:
df_ac_1h = df_ac.resample('H').mean()

In [50]:
df_ac_1h.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 26146 entries, 2019-01-07 08:00:00 to 2021-12-31 17:00:00
Freq: H
Data columns (total 43 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   320PU102           24471 non-null  float64
 1   320PU221           24471 non-null  float64
 2   320RC102DIC02      24471 non-null  float64
 3   330AC102FI06       24471 non-null  float64
 4   330AC102FI07       24471 non-null  float64
 5   330AC102FI08       24471 non-null  float64
 6   330AC102FI09       24471 non-null  float64
 7   330AC102FI10       24471 non-null  float64
 8   330AC102FI51       24471 non-null  float64
 9   330AC102FI52       24471 non-null  float64
 10  330AC102FIC40      24471 non-null  float64
 11  330AC102FICZ01     24471 non-null  float64
 12  330AC102FICZ02     24471 non-null  float64
 13  330AC102FICZ03     24471 non-null  float64
 14  330AC102FICZ04     24471 non-null  float64
 15  330AC102FICZ05     24471 no

Некоторые даты пропущены в данных, уберем пропуски еще раз.

In [51]:
df_ac_1h.dropna(inplace=True)

### Данные исходного ХА по бакам питания

In [52]:
df_chem_in = pd.read_excel(chem_in, index_col=0, parse_dates=True, usecols="B:H")
df_chem_in = df_chem_in.apply(pd.to_numeric, errors='coerce')
df_chem_in.fillna(0, inplace=True)

In [53]:
df_chem_in.head()

,320 RC-101 S Fe,320 RC-101 S Sобщ,320 RC-101 S Sso42-,320 RC-101 S As,320 RC-101 S Сорг,320 RC-101 S Собщ
Timestamp,,,,,,
2018-12-01 09:00:00,27.10,25.4,0.0,8.90,0.47,1.34
2018-12-02 09:00:00,25.99,26.9,0.0,7.95,0.50,1.33
2018-12-02 13:00:00,25.78,27.2,0.0,7.95,0.48,1.32
2018-12-02 17:00:00,26.71,26.2,0.0,8.38,0.43,1.22
2018-12-03 20:00:00,0.00,25.9,0.0,8.79,0.64,1.10


In [54]:
df_chem_in_1h = df_chem_in.resample('H', convention='start').first()

In [55]:
df_chem_in_1h.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 29075 entries, 2018-12-01 09:00:00 to 2022-03-26 19:00:00
Freq: H
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   320 RC-101 S Fe      1502 non-null   float64
 1   320 RC-101 S Sобщ    1502 non-null   float64
 2   320 RC-101 S Sso42-  1502 non-null   float64
 3   320 RC-101 S As      1502 non-null   float64
 4   320 RC-101 S Сорг    1502 non-null   float64
 5   320 RC-101 S Собщ    1502 non-null   float64
dtypes: float64(6)
memory usage: 1.6 MB


In [56]:
cols = ['Fe', 'Stot', 'SO4', 'As', 'Corg', 'Ctot']
df_chem_in_1h.columns = cols

Приведем даты массивов к одному времени. Возьмем ХА только за те дни, что есть данные с АСУТП

In [57]:
min_date = datetime.fromisoformat('2019-01-01 00:00:00')
max_date = datetime.fromisoformat('2021-12-31 17:00:00')

df_chem_in_1h = df_chem_in_1h[min_date:max_date]

Уберем нулевые значения, а вместо них поставим np.nan

In [58]:
df_chem_in_1h.replace({0: np.NaN}, inplace=True)

### Данные ХА по Fe2+

In [59]:
df_chem_fe2 = pd.read_excel(chem_fe2, index_col=0, parse_dates=True, usecols="B,D")
# ispravit na nujnie stolbci
df_chem_fe2 = df_chem_fe2.apply(pd.to_numeric, errors='coerce')
df_chem_fe2.fillna(0, inplace=True)

In [60]:
df_chem_fe2_1h = df_chem_fe2.resample('H').first()

In [61]:
df_chem_fe2_1h.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 29237 entries, 2018-11-29 15:00:00 to 2022-03-31 19:00:00
Freq: H
Data columns (total 1 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   330-ТК-104 L Fe2+  14781 non-null  float64
dtypes: float64(1)
memory usage: 456.8 KB


In [62]:
df_chem_fe2_1h = df_chem_fe2_1h[min_date:max_date]

In [63]:
df_chem_fe2_1h.fillna(method='ffill', inplace=True)

In [64]:
cols = ['Fe2+']
df_chem_fe2_1h.columns = cols

### Данные ОТК по плотностям твердого и питания автоклава¶

In [65]:
path_19 = "C:\\Users\\Laevskiy\\Desktop\\05-22\\Nikitos19.xlsx"
path_20 = "C:\\Users\\Laevskiy\\Desktop\\05-22\\Nikitos20.xlsx"
path_21 = "C:\\Users\\Laevskiy\\Desktop\\05-22\\Nikitos21.xlsx"

ds_19 = pd.read_excel(path_19, sheet_name='AC102', usecols='B:D', index_col=0, parse_dates=True)
ds_20 = pd.read_excel(path_20, sheet_name='AC102', usecols='B:D', index_col=0, parse_dates=True)
ds_21 = pd.read_excel(path_21, sheet_name='AC102', usecols='B:D', index_col=0, parse_dates=True)

In [66]:
ds_full = pd.concat([ds_19, ds_20, ds_21])

In [67]:
assert(ds_19.shape[0] + ds_20.shape[0] + ds_21.shape[0] == ds_full.shape[0])

In [68]:
ds_full = ds_full.apply(pd.to_numeric, errors='coerce')

In [69]:
ds_full_1h = ds_full.resample('H', convention='start').first()

In [70]:
ds_full_1h.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 33133 entries, 2019-01-01 07:00:00 to 2022-10-12 19:00:00
Freq: H
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   D(S)    1304 non-null   float64
 1   D(SL)   1191 non-null   float64
dtypes: float64(2)
memory usage: 776.6 KB


### Финальные проверки массивов данных

In [71]:
print(f'Начальная дата: {min_date}, конечная дата: {max_date}.')

print(f'Данные автоклава от: {df_ac_1h.index.min()} до {df_ac_1h.index.max()}')
print(f'Данные баков питания от: {df_chem_in_1h.index.min()} до {df_chem_in_1h.index.max()}')
print(f'Данные по железу 2+ от: {df_chem_fe2_1h.index.min()} до {df_chem_fe2_1h.index.max()}')
print(f'Данные по плотности в баках питания от: {ds_full_1h.index.min()} до {ds_full_1h.index.max()}')


print(f'Число точек по автоклаву {len(df_ac_1h)}, по бакам питания {len(df_chem_in_1h)}, по железу {len(df_chem_fe2_1h)} и по плотности {len(ds_full_1h)}')

Начальная дата: 2019-01-01 00:00:00, конечная дата: 2021-12-31 17:00:00.
Данные автоклава от: 2019-01-07 08:00:00 до 2021-12-31 17:00:00
Данные баков питания от: 2019-01-01 00:00:00 до 2021-12-31 17:00:00
Данные по железу 2+ от: 2019-01-01 00:00:00 до 2021-12-31 17:00:00
Данные по плотности в баках питания от: 2019-01-01 07:00:00 до 2022-10-12 19:00:00
Число точек по автоклаву 24471, по бакам питания 26298, по железу 26298 и по плотности 33133


### Сохранение данных

In [85]:
file_save(df_ac_1h, WORK_DIR, 'df_ac_1h')

In [86]:
file_save(df_chem_in_1h, WORK_DIR, 'df_chem_in_1h')

In [87]:
file_save(df_chem_fe2_1h, WORK_DIR, 'df_chem_fe2_1h')

In [88]:
file_save(ds_full_1h, WORK_DIR, 'ds_full_1h')

Загрузим ранее сохраненные данные, если не изменений ранее

In [89]:
df_ac_1h = pd.read_csv(f'{WORK_DIR}\df_ac_1h.zip', index_col=0, parse_dates=True)
df_chem_in_1h = pd.read_csv(f'{WORK_DIR}\df_chem_in_1h.zip', index_col=0, parse_dates=True)
df_chem_fe2_1h = pd.read_csv(f'{WORK_DIR}\df_chem_fe2_1h.zip', index_col=0, parse_dates=True)

## Объединение данных в одну таблицу

In [90]:
df1 = df_ac_1h.merge(df_chem_in_1h, how='inner', left_index=True, right_index=True)
df1 = df1.merge(df_chem_fe2_1h, how='inner', left_index=True, right_index=True)
df1 = df1.merge(ds_full_1h, how='inner', left_index=True, right_index=True)

In [91]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 24471 entries, 2019-01-07 08:00:00 to 2021-12-31 17:00:00
Data columns (total 52 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   320PU102           24471 non-null  float64
 1   320PU221           24471 non-null  float64
 2   320RC102DIC02      24471 non-null  float64
 3   330AC102FI06       24471 non-null  float64
 4   330AC102FI07       24471 non-null  float64
 5   330AC102FI08       24471 non-null  float64
 6   330AC102FI09       24471 non-null  float64
 7   330AC102FI10       24471 non-null  float64
 8   330AC102FI51       24471 non-null  float64
 9   330AC102FI52       24471 non-null  float64
 10  330AC102FIC40      24471 non-null  float64
 11  330AC102FICZ01     24471 non-null  float64
 12  330AC102FICZ02     24471 non-null  float64
 13  330AC102FICZ03     24471 non-null  float64
 14  330AC102FICZ04     24471 non-null  float64
 15  330AC102FICZ05     24471 non-null  

In [92]:
df1.head()

,320PU102,320PU221,320RC102DIC02,330AC102FI06,330AC102FI07,330AC102FI08,330AC102FI09,330AC102FI10,330AC102FI51,330AC102FI52,...,OXYGEN/OxyPurityB,Fe,Stot,SO4,As,Corg,Ctot,Fe2+,D(S),D(SL)
2019-01-07 08:00:00,0.0,0.0,988.425903,-0.009983,0.019953,0.000000,-0.002894,0.002894,4950.000488,0.009898,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
2019-01-07 09:00:00,0.0,0.0,988.425903,-0.011309,0.020616,0.000000,-0.002894,0.002894,4950.000488,0.009203,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
2019-01-07 10:00:00,0.0,0.0,988.425903,-0.012683,0.021420,-0.000008,-0.002894,0.002894,4950.000488,0.008668,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
2019-01-07 11:00:00,0.0,0.0,988.425903,-0.011502,0.023771,0.000000,-0.002894,0.002894,4950.000488,0.012060,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
2019-01-07 12:00:00,0.0,0.0,988.425903,-0.010658,0.021340,0.000000,-0.002894,0.002894,4950.000488,0.010702,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN


Сдвинем данные по  𝐹𝑒2+  на 2 часа назад.

In [93]:
df1['Fe2+'] = df1['Fe2+'].shift(-2)

In [94]:
df1.head()

,320PU102,320PU221,320RC102DIC02,330AC102FI06,330AC102FI07,330AC102FI08,330AC102FI09,330AC102FI10,330AC102FI51,330AC102FI52,...,OXYGEN/OxyPurityB,Fe,Stot,SO4,As,Corg,Ctot,Fe2+,D(S),D(SL)
2019-01-07 08:00:00,0.0,0.0,988.425903,-0.009983,0.019953,0.000000,-0.002894,0.002894,4950.000488,0.009898,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
2019-01-07 09:00:00,0.0,0.0,988.425903,-0.011309,0.020616,0.000000,-0.002894,0.002894,4950.000488,0.009203,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
2019-01-07 10:00:00,0.0,0.0,988.425903,-0.012683,0.021420,-0.000008,-0.002894,0.002894,4950.000488,0.008668,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
2019-01-07 11:00:00,0.0,0.0,988.425903,-0.011502,0.023771,0.000000,-0.002894,0.002894,4950.000488,0.012060,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
2019-01-07 12:00:00,0.0,0.0,988.425903,-0.010658,0.021340,0.000000,-0.002894,0.002894,4950.000488,0.010702,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN


In [95]:
file_save(df1, WORK_DIR, 'df_total_ds_1h')